In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
patients =  pd.read_csv('physionet.org/files/mimiciii/1.4/PATIENTS.csv.gz')
cpt =  pd.read_csv('physionet.org/files/mimiciii/1.4/D_CPT.csv.gz')
cpt_event =  pd.read_csv('physionet.org/files/mimiciii/1.4/CPTEVENTS.csv.gz') 
admission = pd.read_csv('physionet.org/files/mimiciii/1.4/ADMISSIONS.csv.gz')
drug =  pd.read_csv('physionet.org/files/mimiciii/1.4/PRESCRIPTIONS.csv.gz')
icu_stay =  pd.read_csv('physionet.org/files/mimiciii/1.4/ICUSTAYS.csv.gz')
ICU_TRANSFER = pd.read_csv('physionet.org/files/mimiciii/1.4/TRANSFERS.csv.gz')
D_LAB = pd.read_csv('physionet.org/files/mimiciii/1.4/D_LABITEMS.csv.gz')
DATETIME_EVENT = pd.read_csv('physionet.org/files/mimiciii/1.4/DATETIMEEVENTS.csv.gz')
D_ITEMS=pd.read_csv('physionet.org/files/mimiciii/1.4/D_ITEMS.csv.gz')
D_ITEMS = D_ITEMS.dropna(subset=['ITEMID','LABEL'])

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
CHARTEVENT = pd.read_csv('physionet.org/files/mimiciii/1.4/CHARTEVENTS.csv.gz') 
input_mv = pd.read_csv('physionet.org/files/mimiciii/1.4/INPUTEVENTS_MV.csv.gz') 
input_cv = pd.read_csv('physionet.org/files/mimiciii/1.4/INPUTEVENTS_CV.csv.gz') 

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,10,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,9,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
D_ICD_diag=pd.read_csv('physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv.gz')
D_ICD_diag = D_ICD_diag.drop(D_ICD_diag.columns[0],axis=1)

D_ICD_proc=pd.read_csv('physionet.org/files/mimiciii/1.4/D_ICD_PROCEDURES.csv.gz', dtype={'ICD9_CODE': str})
D_ICD_proc = D_ICD_proc.drop(D_ICD_proc.columns[0],axis=1)

diag_icd = pd.read_csv('physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv.gz')
diag_icd = diag_icd.drop(diag_icd.columns[0],axis=1)
diag_icd = diag_icd.merge(D_ICD_diag,on='ICD9_CODE')
diag_icd = diag_icd.rename(columns={'SEQ_NUM':'DIAG_SEQ_NUM',
                                    'ICD9_CODE':'DIA_ICD9_CODE','SHORT_TITLE':'DIAG_SHORT_TITLE',
                                   'LONG_TITLE':'DIAG_LONG_TITLE'})

def process_code(code):
    if code.startswith('E') or code.startswith('V'):
        return code[:1]
    else:
        return code[:3]

diag_icd['ICD9'] = diag_icd['DIA_ICD9_CODE'].apply(process_code)

proc_icd = pd.read_csv('physionet.org/files/mimiciii/1.4/PROCEDURES_ICD.csv.gz',dtype={'ICD9_CODE': str})
proc_icd = proc_icd.drop(proc_icd.columns[0],axis=1)
proc_icd = proc_icd.merge(D_ICD_proc,on='ICD9_CODE')
proc_icd = proc_icd.rename(columns={'SEQ_NUM':'PROC_SEQ_NUM','ICD9_CODE':'PROC_ICD9_CODE',
                                   'SHORT_TITLE':'PROC_SHORT_TITLE',
                                   'LONG_TITLE':'PROC_LONG_TITLE'})

In [5]:
### Define Cohort

In [5]:
service = pd.read_csv('physionet.org/files/mimiciii/1.4/SERVICES.csv.gz')


surg_service = service[service.CURR_SERVICE.str.contains('SURG')| 
        (service.CURR_SERVICE.isin(["ORTHO"]))]


group = surg_service.groupby('HADM_ID',as_index=False).CURR_SERVICE.nunique()
multi_surgery = group[group.CURR_SERVICE>1]
single_surgery = group[group.CURR_SERVICE==1]

single_surg = surg_service[surg_service.HADM_ID.isin(single_surgery.HADM_ID)]

# surgery_hadm = surg_service.HADM_ID.unique()
# surgery_subject = surg_service.SUBJECT_ID.unique()
surgery_hadm = single_surg.HADM_ID.unique()
surgery_subject = single_surg.SUBJECT_ID.unique()


def MapHADM(table,hadmid=surgery_hadm):
    table1 = table[table.HADM_ID.isin(hadmid)]
    return(table1)

                
    
icu_surg = MapHADM(icu_stay).drop(['ROW_ID'],axis=1)     

surg_CHARTEVENT = CHARTEVENT[CHARTEVENT.HADM_ID.isin(surgery_hadm)]
#surg_CHARTEVENT.to_csv("surg_CHARTEVENT.csv")

# surgery_subject = surg_CHARTEVENT.SUBJECT_ID.unique()
# surgery_hadm = surg_CHARTEVENT.HADM_ID.unique()

len(surgery_hadm)


20065

In [6]:
def raw_data():

    
    ##    demographic
    admission_surg = admission[admission.HADM_ID.isin(surgery_hadm)]
    demo = admission_surg.drop(['ROW_ID','HOSPITAL_EXPIRE_FLAG','HOSPITAL_EXPIRE_FLAG'],axis=1)
    demo = demo.merge(patients.drop(['ROW_ID'],axis=1),on='SUBJECT_ID',how='left')

    
    
    # Get Age
    DOB = [datetime.datetime.strptime(demo.DOB.values[i], "%Y-%m-%d %H:%M:%S").date() for i in range(demo.shape[0])]
    DOA = [datetime.datetime.strptime(demo.ADMITTIME.values[i], "%Y-%m-%d %H:%M:%S").date() for i in range(demo.shape[0])]
    age = pd.Series([round((DOA[i] -DOB[i]).days/365) for i in range(demo.shape[0])])
    demo['AGE'] = age
    demo.insert(3, 'AGE', demo.pop('AGE'))
    demo.insert(6, 'INSURANCE', demo.pop('INSURANCE'))
    demo.insert(7, 'GENDER', demo.pop('GENDER'))
    demo.insert(8,'ETHNICITY',demo.pop('ETHNICITY'))
    demo.insert(9, 'RELIGION',demo.pop('RELIGION'))
    demo.insert(10,'MARITAL_STATUS',demo.pop('MARITAL_STATUS'))
    
    hadm_record = demo
    
    
    
    
    ## ICU
    icu_info = icu_surg[['SUBJECT_ID','HADM_ID',
                     'ICUSTAY_ID','LOS','INTIME',
                     'OUTTIME','FIRST_CAREUNIT',
                     'LAST_CAREUNIT']].sort_values('HADM_ID').rename(columns={'LOS':'LOS(Day)'})
    icu_surg_agg = icu_info.groupby(['SUBJECT_ID','HADM_ID'],as_index=False).agg({'LOS(Day)':'sum',
                                                                                'INTIME':'first','OUTTIME':'last',
                                                                                            'FIRST_CAREUNIT':'first',
                                                                                         'LAST_CAREUNIT':'last'})
    hadm_record = demo.merge(icu_surg_agg,on=['SUBJECT_ID','HADM_ID'],how="left")
    
    
    
    
    
    ## weight
    weight = surg_CHARTEVENT[surg_CHARTEVENT.ITEMID.isin([762, 763, 
                                                      3580, 3581, 3582,3693,
                                                      224639,226512,226531])]
    weight = weight[weight.VALUENUM<700]
    weight = weight[weight.VALUENUM>0]
    weight = weight.merge(D_ITEMS[['ITEMID','LABEL']],on='ITEMID',how='left')
    weight.loc[weight['LABEL']=='Present Weight  (lb)', 'VALUENUM'] *= 0.45359237
    weight.loc[weight['LABEL']=='Present Weight  (lb)', 'VALUEUOM'] = 'kg'
    weight.loc[weight['LABEL']=='Present Weight  (oz)', 'VALUENUM'] *= 0.02834952
    weight.loc[weight['LABEL']=='Present Weight  (oz)', 'VALUEUOM'] = 'kg'
    weight.loc[weight['VALUEUOM']=='gms', 'VALUEUOM'] = 'kg' 
    weight = weight.dropna(subset=['VALUEUOM']).sort_values(by='CHARTTIME')
    # weight.rename(columns={'LABEL':'Weight_Label'})
    weight = weight.rename(columns={'VALUENUM':'Weight','LABEL':'Weight_Label'})
    weight_stat=weight.groupby('HADM_ID',as_index=False).agg({'SUBJECT_ID':'first',
                                                        'VALUEUOM':'first','Weight':['mean','min','max','std','size']})
    weight_stat.columns = [f'{col[0]}_{col[1]}' for col in weight_stat.columns]
    weight_stat = weight_stat.rename(columns={'HADM_ID_':'HADM_ID','SUBJECT_ID_first':'SUBJECT_ID',
                             'VALUEUOM_first':'WeightUnit','Weight_size':'Weight_N'})
    hadm_record = hadm_record.merge(weight_stat,on=['SUBJECT_ID','HADM_ID'],how='left')
    
    
    
    
    
    ## Height
    Height = surg_CHARTEVENT[surg_CHARTEVENT.ITEMID.isin(D_ITEMS[D_ITEMS.ITEMID.isin([920, 1394, 4187,
                                                                                  3486, 3485, 4188,
                                                                                 226707,226730])].ITEMID)]
#     Height = Height.dropna(subset=['VALUE','VALUENUM'])
    height = Height[Height.VALUENUM>0]
    height = height.merge(D_ITEMS[['ITEMID','LABEL']], on='ITEMID',how='left')
    height.loc[height['VALUEUOM'].isin(['Inch', 'Inches','inches']), 'VALUENUM'] *= 2.54
    height.loc[height['VALUEUOM'].isin(['Inch', 'Inches','inches']), 'VALUEUOM'] = 'cm'
    height = height.rename(columns={'VALUENUM':'Height'})
    stat = height.groupby('HADM_ID',as_index=False).agg({'SUBJECT_ID':'first','VALUEUOM':'first',
                                                         'Height':['mean','min','max','std','size']})
    stat.columns = [f'{col[0]}_{col[1]}' for col in stat.columns]
    stat = stat.rename(columns={'HADM_ID_':'HADM_ID','SUBJECT_ID_first':'SUBJECT_ID',
                            'VALUEUOM_first':'HeightUnit','Height_size':'Height_N'})
    hadm_record = hadm_record.merge(stat,on=['SUBJECT_ID','HADM_ID'],how='left')
    hadm_record["BMI"]= 10000*hadm_record['Weight_mean']/(hadm_record['Height_mean']**2)

    
    
    ## Pain Score
    PAIN=surg_CHARTEVENT[surg_CHARTEVENT['ITEMID'].isin([524,1044,227881,223791,225813])] #224409, 1045
    pain_acceptable = surg_CHARTEVENT[surg_CHARTEVENT['ITEMID'].isin([1089,223794])]
    pain = PAIN.merge(D_ITEMS[['ITEMID','LABEL']],on='ITEMID',how='left')
    mapping = {
    '0 / 10':0,
    '0-None': 0,
    '1-None to Mild':1,
    '2 / 10' : 2,
    '2-Mild':2,
    '3-Mild to Mod':3,
    '4-Mild to Mod':4,    
    '5 / 10':5,
    '5-Moderate':5,
    '6-Mod to Severe':6,
    '7-Mod to Severe':7,
    '8 / 10': 8,
    '8-Severe': 8,
    '9-Severe-Worst':9,
    '10 / 10': 10,
    '10-Worst': 10,
    'Unable to Score': None,
    'Unable to score': None}
    
    pain.loc[pain['VALUE'].isin(['Unable to score','Unable to Score']), 'VALUENUM'] = None
    for label in mapping.keys():
        pain.loc[pain.VALUE == label, 'VALUENUM'] = mapping[label]
        
    pain = pain.dropna(subset=['VALUE']) 
    pain_score = pain[pain.LABEL!='Baseline pain level'].dropna(subset=['VALUENUM'])
#     pain_score.to_csv("detailed pain score records_by_time.csv")
    pain_score = pain_score.rename(columns={'VALUENUM':'PainScore'})
    pain_stat = pain_score.groupby('HADM_ID',as_index=False).agg({'PainScore':['mean',
                                                                'max', 'min','std','count'],'SUBJECT_ID':'first'})
    pain_stat.columns = [f'{col[0]}_{col[1]}' for col in pain_stat.columns]
    pain_stat = pain_stat.rename(columns={'HADM_ID_':'HADM_ID','SUBJECT_ID_first':'SUBJECT_ID'})
    hadm_record = hadm_record.merge(pain_stat,on=['SUBJECT_ID','HADM_ID'],how='left')

    
    
    
    ## Baseline Pain
    baseline = pain.groupby('LABEL').get_group('Baseline pain level')
    baseline = baseline.rename(columns={'VALUENUM':'BaselinePainScore','VALUE':'BaselinePain'})
    baseline.loc[baseline.BaselinePain.isin(['Unable to Score','Unable to score']), 'BaselinePainScore'] = None
    baseline = baseline.dropna(subset=['BaselinePainScore'])
    baseline_stat = baseline.groupby('HADM_ID',as_index=False).agg({'BaselinePainScore':['mean','max','min','std','size'],
                                                     'SUBJECT_ID':'first'})
    baseline_stat.columns = [f'{col[0]}_{col[1]}' for col in baseline_stat.columns]
    baseline_stat = baseline_stat.rename(columns={'HADM_ID_':'HADM_ID','SUBJECT_ID_first':'SUBJECT_ID',
                                                 'BaselinePainScore_size':'BaselinePainScore_N'})
    hadm_record = hadm_record.merge(baseline_stat,on=['SUBJECT_ID','HADM_ID'],how='left')
    
    
    
    
#     Surgery type
#     surg_type = cpt_event[cpt_event.HADM_ID.isin(surgery_hadm)][['HADM_ID','CHARTDATE',
#                                                                         'SECTIONHEADER','SUBSECTIONHEADER']]
#     surg_type = surg_type[surg_type.SECTIONHEADER=='Surgery']
#     records = hadm_record.merge(surg_type,on='HADM_ID')
#     records = records.drop_duplicates(subset=['HADM_ID'], keep='first')
    surg_type = surg_service[surg_service.HADM_ID.isin(surgery_hadm)]
    records = hadm_record.merge(surg_type[['HADM_ID','CURR_SERVICE']],on='HADM_ID',how='left')
    surg_map = {'VSURG':'Circulatory',
              'CSURG':'Cardiac',
              'NSURG':'Neurologic',
              'ORTHO':'Musculoskeletal',
              'PSURG':'Plastic',
              'SURG':'General',
              'TSURG':'Thoracic'}
    for label in surg_map.keys():
        records.loc[records.CURR_SERVICE == label, 'Surgery'] = surg_map[label]


      ## Cormorbidity
    records_diag = records[['SUBJECT_ID','HADM_ID',
                       'Surgery']].merge(diag_icd,on=['HADM_ID','SUBJECT_ID'],how='left')
    records_diag = records_diag.sort_values(by=['HADM_ID','DIAG_SEQ_NUM'])
    unique_diag_long_titles = records_diag[["DIAG_LONG_TITLE",'DIA_ICD9_CODE']].drop_duplicates()
    unique_diag_long_titles_df = pd.DataFrame(unique_diag_long_titles, columns=["DIAG_LONG_TITLE",'DIA_ICD9_CODE'])
   
    
    
    
    # Morbidities_N':
    diag_summary = records_diag.groupby('HADM_ID',as_index=False).agg({'SUBJECT_ID':'first','DIAG_LONG_TITLE':'size',
                                     'Surgery':'first'}).rename(columns={'DIAG_LONG_TITLE':'Morbidities_N'})
    hadm_record = hadm_record.merge(diag_summary,on=['HADM_ID','SUBJECT_ID'],how='left')
    
    
    
    
    ## Mortarlity
    icu_n = icu_stay[icu_stay.HADM_ID.isin(surgery_hadm)].groupby('HADM_ID',
                                                                         as_index=False).agg({'ICUSTAY_ID':'nunique'}).rename(columns={'ICUSTAY_ID':'ICUSTAY_ADMIT_N'})
    hadm_record = hadm_record.merge(icu_n,on='HADM_ID',how='left')
    hadm_record.insert(24, 'ICUSTAY_ADMIT_N', hadm_record.pop('ICUSTAY_ADMIT_N'))
    hadm_record['MORTARLITY'] = 0
    # hadm_record2.loc[hadm_record2.DOD.notna(),'MORTARLITY'] = 1 
    hadm_record.loc[hadm_record.DOD<=hadm_record.DISCHTIME,'MORTARLITY'] = 1
    
    
    
    return records_diag, hadm_record


In [ ]:
# data merge

In [7]:
def merge_type(X):
    X['ETHNICITY'] = X['ETHNICITY'].replace(['AMERICAN INDIAN/ALASKA NATIVE',
                                         'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE'] ,
                                        'AMERICAN INDIAN/ALASKA NATIVE')


    X['ETHNICITY'] = X['ETHNICITY'].replace(['ASIAN', 'ASIAN - THAI',
                                             'ASIAN - ASIAN INDIAN', 
                                             'ASIAN - CAMBODIAN', 'ASIAN - CHINESE',
                                             'ASIAN - FILIPINO', 'ASIAN - JAPANESE', 'ASIAN - KOREAN', 
                                             'ASIAN - OTHER', 'ASIAN - VIETNAMESE'], 'ASIAN')


    X['ETHNICITY'] = X['ETHNICITY'].replace(['BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 
                                             'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN'], 'BLACK/AFRICAN')


    X['ETHNICITY'] = X['ETHNICITY'].replace(['CARIBBEAN ISLAND','NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                                            'MIDDLE EASTERN','AMERICAN INDIAN/ALASKA NATIVE',
                                             'MULTI RACE ETHNICITY','OTHER',
                                             'PORTUGUESE', 'SOUTH AMERICAN','ASIAN'],'OTHER SPECIES')

    X['ETHNICITY'] = X['ETHNICITY'].replace(['HISPANIC OR LATINO', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 
                                             'HISPANIC/LATINO - COLOMBIAN', 'HISPANIC/LATINO - CUBAN',
                                             'HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO - GUATEMALAN',
                                             'HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO - MEXICAN', 
                                             'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - SALVADORAN'], 
                                            'HISPANIC/LATINO')

    X['ETHNICITY'] = X['ETHNICITY'].replace(['PATIENT DECLINED TO ANSWER', 
                                             'UNABLE TO OBTAIN', 'UNKNOWN/NOT SPECIFIED'], 'UNKNOWN')


    X['ETHNICITY'] = X['ETHNICITY'].replace(['PORTUGUESE', 'SOUTH AMERICAN'],'SOUTH AMERICAN/PORTUGUESE')
    X['ETHNICITY'] = X['ETHNICITY'].replace(['WHITE', 'WHITE - BRAZILIAN', 'WHITE - EASTERN EUROPEAN',
                                             'WHITE - OTHER EUROPEAN', 'WHITE - RUSSIAN'], 'WHITE')



    X['MARITAL_STATUS'] = X['MARITAL_STATUS'].replace(['LIFE PARTNER', 'UNKNOWN (DEFAULT)',None],'Other')
    X['MARITAL_STATUS'] = X['MARITAL_STATUS'].replace(['DIVORCED', 'SEPARATED'],'DIVORCED/SEPARATED')



    X['INSURANCE'] = X['INSURANCE'].replace(['Self Pay', 'Private'],'Self-Pay/Private')
    X['INSURANCE'] = X['INSURANCE'].replace(['Medicare', 'Government'],'Medicare/Government')
    
    X['ADMISSION_LOCATION'] = X['ADMISSION_LOCATION'].replace(['** INFO NOT AVAILABLE **',
                                                              'HMO REFERRAL/SICK',
                                                              'TRANSFER FROM OTHER HEALT',
                                                              'TRANSFER FROM SKILLED NUR'],'Other')
    
    
    
    return X


records_diag, hadm_record  = raw_data()

In [8]:
records_diag = records_diag.drop_duplicates().dropna()
records_diag

,SUBJECT_ID,HADM_ID,Surgery,DIAG_SEQ_NUM,DIA_ICD9_CODE,DIAG_SHORT_TITLE,DIAG_LONG_TITLE,ICD9
73678,23018,100007,General,1.0,56081,Intestinal adhes w obstr,Intestinal or peritoneal adhesions with obstru...,560
73677,23018,100007,General,2.0,5570,Ac vasc insuff intestine,Acute vascular insufficiency of intestine,557
73675,23018,100007,General,4.0,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",486
73676,23018,100007,General,5.0,4019,Hypertension NOS,Unspecified essential hypertension,401
2259,533,100009,Cardiac,1.0,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...,414
...,...,...,...,...,...,...,...,...
90703,27200,199998,Cardiac,12.0,4439,Periph vascular dis NOS,"Peripheral vascular disease, unspecified",443
90705,27200,199998,Cardiac,13.0,2724,Hyperlipidemia NEC/NOS,Other and unspecified hyperlipidemia,272
90708,27200,199998,Cardiac,14.0,51889,Other lung disease NEC,"Other diseases of lung, not elsewhere classified",518
90698,27200,199998,Cardiac,15.0,2720,Pure hypercholesterolem,Pure hypercholesterolemia,272


In [9]:
#basic demographic information
X = merge_type(hadm_record)
X.to_csv('X.csv',index=False)
X

,SUBJECT_ID,HADM_ID,ADMITTIME,AGE,DISCHTIME,DEATHTIME,INSURANCE,GENDER,ETHNICITY,RELIGION,...,PainScore_std,PainScore_count,BaselinePainScore_mean,BaselinePainScore_max,BaselinePainScore_min,BaselinePainScore_std,BaselinePainScore_N,Morbidities_N,Surgery,MORTARLITY
0,23,152223,2153-09-03 07:15:00,71,2153-09-08 19:10:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,1.851640,8.0,NaN,NaN,NaN,NaN,NaN,8,Cardiac,0
1,23,124321,2157-10-18 19:34:00,75,2157-10-25 14:00:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,1.414214,2.0,NaN,NaN,NaN,NaN,NaN,10,Neurologic,0
2,28,162569,2177-09-01 07:15:00,74,2177-09-06 16:00:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,0.000000,2.0,NaN,NaN,NaN,NaN,NaN,10,Cardiac,0
3,32,175413,2170-04-04 08:00:00,46,2170-04-23 12:45:00,NaN,Medicaid,M,WHITE,UNOBTAINABLE,...,1.406422,14.0,NaN,NaN,NaN,NaN,NaN,6,Thoracic,0
4,35,166707,2122-02-10 11:15:00,69,2122-02-20 15:30:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,1.138990,38.0,NaN,NaN,NaN,NaN,NaN,11,Cardiac,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20060,98761,182540,2186-02-08 12:30:00,47,2186-02-08 20:21:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,General,0
20061,98769,141860,2125-02-22 22:18:00,80,2125-03-05 17:22:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,2.633609,13.0,NaN,NaN,NaN,NaN,NaN,15,General,0
20062,98790,187987,2200-10-15 01:58:00,54,2200-10-25 15:43:00,NaN,Self-Pay/Private,M,WHITE,NOT SPECIFIED,...,1.705606,22.0,NaN,NaN,NaN,NaN,NaN,10,Musculoskeletal,0
20063,98794,190603,2127-11-07 11:00:00,78,2127-11-11 14:30:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,1.643168,5.0,NaN,NaN,NaN,NaN,NaN,5,Cardiac,0


In [ ]:
# Define comorbidities 

In [ ]:
X = pd.read_csv('X.csv')

In [10]:
def morb():
    hadm = X.HADM_ID.unique()
    df = pd.DataFrame(records_diag)
    icd9_names = {
        '305': 'Alcohol or drug abuse', '303': 'Alcohol or drug abuse','304': 'Alcohol or drug abuse',
        '410': 'Myocardial Infarction (heart attack)',
        '428': 'Congestive Heart Failure',    
        '430': 'Cerebrovascular Disease (stroke)','431': 'Cerebrovascular Disease (stroke)','432': 'Cerebrovascular Disease (stroke)','433': 'Cerebrovascular Disease (stroke)','434': 'Cerebrovascular Disease (stroke)','435': 'Cerebrovascular Disease (stroke)','436': 'Cerebrovascular Disease (stroke)','437': 'Cerebrovascular Disease (stroke)','438': 'Cerebrovascular Disease (stroke)',
        '490': 'Chronic Pulmonary Disease','491': 'Chronic Pulmonary Disease','492': 'Chronic Pulmonary Disease','493': 'Chronic Pulmonary Disease','494': 'Chronic Pulmonary Disease','495': 'Chronic Pulmonary Disease','496': 'Chronic Pulmonary Disease',
        '140': 'Cancer','141': 'Cancer', '142': 'Cancer','143': 'Cancer','144': 'Cancer','145': 'Cancer','146': 'Cancer','147': 'Cancer','148': 'Cancer','149': 'Cancer','150': 'Cancer','151': 'Cancer','152': 'Cancer','153': 'Cancer','154': 'Cancer','155': 'Cancer','156': 'Cancer','157': 'Cancer','158': 'Cancer','159': 'Cancer','160': 'Cancer','161': 'Cancer','162': 'Cancer','163': 'Cancer',
        '164': 'Cancer','165': 'Cancer','166': 'Cancer','167': 'Cancer','168': 'Cancer','169': 'Cancer','170': 'Cancer','171': 'Cancer','172': 'Cancer','173': 'Cancer','174': 'Cancer','175': 'Cancer','176': 'Cancer','177': 'Cancer', '178': 'Cancer','179': 'Cancer','180': 'Cancer','181': 'Cancer','182': 'Cancer','183': 'Cancer','184': 'Cancer','185': 'Cancer','186': 'Cancer','187': 'Cancer','188': 'Cancer', '189': 'Cancer','190': 'Cancer','191': 'Cancer', '192': 'Cancer','193': 'Cancer','194': 'Cancer', '195': 'Cancer',
        '196': 'Metastatic Carcinoma','197': 'Metastatic Carcinoma', '198': 'Metastatic Carcinoma','199': 'Metastatic Carcinoma',
        '570': 'Liver Disease','571': 'Liver Disease','572': 'Liver Disease','573': 'Liver Disease',
        '250': 'Diabetes',
        '401': 'Hypertension','402': 'Hypertension','403': 'Hypertension','404': 'Hypertension','405': 'Hypertension',
        '278': 'Obesity',
        '276': 'Fluid and electrolyte disorders',
        '394': 'Valvular Disease','395': 'Valvular Disease','396': 'Valvular Disease','397': 'Valvular Disease',
        '286': 'Coagulopathy',    
        '311': 'Depression',    
        '585': 'Chronic Kidney Disease' 
    }

    icd9_names2={
    '2859': 'Chronic anemia','78321': 'Weight Loss',
        '4439': 'Peripheral Vascular Disease','5856': 'End Stage Renal Disease'}

    # Create a new column 'ICD9_NAME' based on the mapping
    df['ICD9_NAME'] = df['ICD9'].map(icd9_names)
    df['ICD9_NAME2'] = df['DIA_ICD9_CODE'].map(icd9_names2)
    df['ICD9_NAME'].fillna(df['ICD9_NAME2'], inplace=True)
    df.drop('ICD9_NAME2', axis=1, inplace=True)
    df = df.rename(columns={"ICD9_NAME":"Cormor"})
    print(df.head())
    
    morbidities = pd.get_dummies(df[['HADM_ID','Cormor']], columns=['Cormor'])
    morbidities = morbidities.groupby('HADM_ID').max().reset_index()
    
    summary = df.groupby('Cormor').agg({'HADM_ID':'nunique'}).sort_values(by=['HADM_ID'],ascending=False)
    summary = summary.rename(columns={'HADM_ID':'Frequency'})
    summary['Sample Size'] = len(hadm)
    summary['Percent(%)'] = 100 * summary.Frequency / len(hadm)
    summary
    
    return morbidities, summary, df

In [11]:
morb,morb_summary,df = morb()

       SUBJECT_ID  HADM_ID  Surgery  DIAG_SEQ_NUM DIA_ICD9_CODE  \
73678       23018   100007  General           1.0         56081   
73677       23018   100007  General           2.0          5570   
73675       23018   100007  General           4.0           486   
73676       23018   100007  General           5.0          4019   
2259          533   100009  Cardiac           1.0         41401   

               DIAG_SHORT_TITLE  \
73678  Intestinal adhes w obstr   
73677  Ac vasc insuff intestine   
73675   Pneumonia, organism NOS   
73676          Hypertension NOS   
2259   Crnry athrscl natve vssl   

                                         DIAG_LONG_TITLE ICD9        Cormor  
73678  Intestinal or peritoneal adhesions with obstru...  560           NaN  
73677          Acute vascular insufficiency of intestine  557           NaN  
73675                    Pneumonia, organism unspecified  486           NaN  
73676                 Unspecified essential hypertension  401  Hypertensio

In [12]:
morb.to_csv('morb.csv',index=False)

In [13]:
morb

,HADM_ID,Cormor_Alcohol or drug abuse,Cormor_Cancer,Cormor_Cerebrovascular Disease (stroke),Cormor_Chronic Kidney Disease,Cormor_Chronic Pulmonary Disease,Cormor_Chronic anemia,Cormor_Coagulopathy,Cormor_Congestive Heart Failure,Cormor_Depression,Cormor_Diabetes,Cormor_Fluid and electrolyte disorders,Cormor_Hypertension,Cormor_Liver Disease,Cormor_Metastatic Carcinoma,Cormor_Myocardial Infarction (heart attack),Cormor_Obesity,Cormor_Peripheral Vascular Disease,Cormor_Valvular Disease,Cormor_Weight Loss
0,100007,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,100009,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
2,100012,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,100014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,100018,0,0,1,1,1,0,0,1,0,1,1,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,199979,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
20054,199988,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20055,199993,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0
20056,199995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# Separate DataFrames for External Causes and Health Status categories
e = records_diag[records_diag.ICD9.str.contains("E")].copy()
e['Category'] = 'E: External Causes of Injury and Poisoning'

v = records_diag[records_diag.ICD9.str.contains("V")].copy()
v['Category'] = 'V: Factors Influencing Health Status and Contact with Health Services'

# Define ICD-9-CM category boundaries
bins = [1, 139, 239, 279, 289, 319, 389, 459, 519, 579, 629, 679, 709, 739, 759, 779, 799, 999]
labels = [
    'Infectious and Parasitic Diseases',  # 1-139
    'Neoplasms',  # 140-239
    'Metabolic, Endocrine, and Nutritional Diseases',  # 240-279
    'Blood and Blood-Forming Organs Diseases',  # 280-289
    'Mental Disorder',  # 290-319
    'Nervous System and Sense Organs Diseases',  # 320-389
    'Circulatory System Diseases',  # 390-459
    'Respiratory System Diseases',  # 460-519
    'Digestive System Diseases',  # 520-579
    'Genitourinary System Diseases',  # 580-629
    'Pregnancy, Childbirth, and the Puerperium',  # 630-679
    'Skin and Subcutaneous Tissue Diseases',  # 680-709
    'Musculoskeletal System and Connective Tissue Diseases',  # 710-739
    'Congenital Anomalies',  # 740-759
    'Certain Conditions Originating in the Perinatal Period',  # 760-779
    'Symptoms, Signs, and Ill-Defined Conditions',  # 780-799
    'Injury and Poisoning'  # 800-999
]

# Classify ICD-9 codes into different categories
df = records_diag[~records_diag.ICD9.str.contains("E|V")].copy()
df['Category'] = pd.cut(df['ICD9'].astype(int), bins=bins, labels=labels)

# Combine DataFrames
df = pd.concat([df, e, v], axis=0)
df = df.sort_values(by=['HADM_ID', 'DIAG_SEQ_NUM'])

# Define mappings for specific ICD-9 codes to their names
icd9_names = {
    '305': 'Alcohol or drug abuse', '303': 'Alcohol or drug abuse','304': 'Alcohol or drug abuse',
    '410': 'Myocardial Infarction (heart attack)',
    '428': 'Congestive Heart Failure',    
    '430': 'Cerebrovascular Disease (stroke)','431': 'Cerebrovascular Disease (stroke)','432': 'Cerebrovascular Disease (stroke)','433': 'Cerebrovascular Disease (stroke)','434': 'Cerebrovascular Disease (stroke)','435': 'Cerebrovascular Disease (stroke)','436': 'Cerebrovascular Disease (stroke)','437': 'Cerebrovascular Disease (stroke)','438': 'Cerebrovascular Disease (stroke)',
    '490': 'Chronic Pulmonary Disease','491': 'Chronic Pulmonary Disease','492': 'Chronic Pulmonary Disease','493': 'Chronic Pulmonary Disease','494': 'Chronic Pulmonary Disease','495': 'Chronic Pulmonary Disease','496': 'Chronic Pulmonary Disease',
    '140': 'Cancer','141': 'Cancer', '142': 'Cancer','143': 'Cancer','144': 'Cancer','145': 'Cancer','146': 'Cancer','147': 'Cancer','148': 'Cancer','149': 'Cancer','150': 'Cancer','151': 'Cancer','152': 'Cancer','153': 'Cancer','154': 'Cancer','155': 'Cancer','156': 'Cancer','157': 'Cancer','158': 'Cancer','159': 'Cancer','160': 'Cancer','161': 'Cancer','162': 'Cancer','163': 'Cancer',
    '164': 'Cancer','165': 'Cancer','166': 'Cancer','167': 'Cancer','168': 'Cancer','169': 'Cancer','170': 'Cancer','171': 'Cancer','172': 'Cancer','173': 'Cancer','174': 'Cancer','175': 'Cancer','176': 'Cancer','177': 'Cancer', '178': 'Cancer','179': 'Cancer','180': 'Cancer','181': 'Cancer','182': 'Cancer','183': 'Cancer','184': 'Cancer','185': 'Cancer','186': 'Cancer','187': 'Cancer','188': 'Cancer', '189': 'Cancer','190': 'Cancer','191': 'Cancer', '192': 'Cancer','193': 'Cancer','194': 'Cancer', '195': 'Cancer',
    '196': 'Metastatic Carcinoma','197': 'Metastatic Carcinoma', '198': 'Metastatic Carcinoma','199': 'Metastatic Carcinoma',
    '570': 'Liver Disease','571': 'Liver Disease','572': 'Liver Disease','573': 'Liver Disease',
    '250': 'Diabetes',
    '401': 'Hypertension','402': 'Hypertension','403': 'Hypertension','404': 'Hypertension','405': 'Hypertension',
    '278': 'Obesity',
    '276': 'Fluid and electrolyte disorders',
    '394': 'Valvular Disease','395': 'Valvular Disease','396': 'Valvular Disease','397': 'Valvular Disease',
    '286': 'Coagulopathy',    
    '311': 'Depression',    
    '585': 'Chronic Kidney Disease',   
}

icd9_names2 = {
    '2859': 'Chronic anemia',
    '78321': 'Weight Loss',
    '4439': 'Peripheral Vascular Disease',
    '5856': 'End Stage Renal Disease'
}

# Map ICD-9 codes to their names
df['ICD9_NAME'] = df['ICD9'].map(icd9_names)
df['ICD9_NAME2'] = df['DIA_ICD9_CODE'].map(icd9_names2)

# Fill missing values in 'ICD9_NAME' with values from 'ICD9_NAME2'
df['ICD9_NAME'].fillna(df['ICD9_NAME2'], inplace=True)

df.drop(['Surgery', 'ICD9_NAME2'], axis=1, inplace=True)

In [15]:
df.to_csv('ICD9_category.csv',index=False)

In [16]:
df

,SUBJECT_ID,HADM_ID,DIAG_SEQ_NUM,DIA_ICD9_CODE,DIAG_SHORT_TITLE,DIAG_LONG_TITLE,ICD9,ICD9_NAME,Category
73678,23018,100007,1.0,56081,Intestinal adhes w obstr,Intestinal or peritoneal adhesions with obstru...,560,NaN,Digestive System Diseases
73677,23018,100007,2.0,5570,Ac vasc insuff intestine,Acute vascular insufficiency of intestine,557,NaN,Digestive System Diseases
73675,23018,100007,4.0,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",486,NaN,Respiratory System Diseases
73676,23018,100007,5.0,4019,Hypertension NOS,Unspecified essential hypertension,401,Hypertension,Circulatory System Diseases
2259,533,100009,1.0,41401,Crnry athrscl natve vssl,Coronary atherosclerosis of native coronary ar...,414,NaN,Circulatory System Diseases
...,...,...,...,...,...,...,...,...,...
90703,27200,199998,12.0,4439,Periph vascular dis NOS,"Peripheral vascular disease, unspecified",443,Peripheral Vascular Disease,Circulatory System Diseases
90705,27200,199998,13.0,2724,Hyperlipidemia NEC/NOS,Other and unspecified hyperlipidemia,272,NaN,"Metabolic, Endocrine, and Nutritional Diseases"
90708,27200,199998,14.0,51889,Other lung disease NEC,"Other diseases of lung, not elsewhere classified",518,NaN,Respiratory System Diseases
90698,27200,199998,15.0,2720,Pure hypercholesterolem,Pure hypercholesterolemia,272,NaN,"Metabolic, Endocrine, and Nutritional Diseases"


In [ ]:
# Postoperative Infection1 (ICD9)

In [ ]:
df=pd.read_csv('ICD9_category.csv')

In [17]:
#Relative ICD9 codes 
v2_codes1=['9985', '99851', '99859']# postoperative infection
v2_codes2=['6743'] #Postoperative obstetrical wound infection
v2_codes3=[ '010', '011', '012', '013', '014', '015', '016', '017', '018', '030', 
            '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', ]#Subset Bacterial Infections 001-139
v2_codes4=['9966','99660', '99661', '99662', '99663', '99664', '99665', '99666', '99667', '99668', '99669']#Infection and inflammatory reaction due to internal prosthetic device implant and graft: 996.60-996.69

In [18]:
##postoperative infection
v2_rows1 = df[df['DIA_ICD9_CODE'].isin(v2_codes1)]
#Postoperative obstetrical wound infection
v2_rows2 = df[df['DIA_ICD9_CODE'].isin(v2_codes2)]
#Bacterial Infections
v2_rows3 = df[df['ICD9'].isin(v2_codes3)]
#Infection and inflammatory reaction due to internal prosthetic device implant and graft
v2_rows4 = df[df['DIA_ICD9_CODE'].isin(v2_codes4)]

dataframes_v2 = [v2_rows1,v2_rows2,v2_rows3,v2_rows4]
infection2_ICD9=pd.concat(dataframes_v2,axis=0)
infection2_ICD9=infection2_ICD9.reset_index(drop=True)

In [19]:
#annotated relative DIAG_LONG_TITLE
annot_title=['Unspecified septicemia','Infection and inflammatory reaction due to other vascular device, implant, and graft',
             'Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site',
             'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
             'Pseudomonas infection in conditions classified elsewhere and of unspecified site','Methicillin susceptible Staphylococcus aureus septicemia',
             'Methicillin resistant Staphylococcus aureus in conditions classified elsewhere and of unspecified site','Septicemia due to escherichia coli [E. coli]',
             'Other specified bacterial infections in conditions classified elsewhere and of unspecified site, other gram-negative organisms','Other septicemia due to gram-negative organisms',
             'Staphylococcus infection in conditions classified elsewhere and of unspecified site, other staphylococcus','Streptococcal septicemia','Methicillin resistant Staphylococcus aureus septicemia',
             'Other specified septicemias','Streptococcus infection in conditions classified elsewhere and of unspecified site, other streptococcus',
             'Infection and inflammatory reaction due to cardiac device, implant, and graft','Other and unspecified Escherichia coli [E. coli]','Septicemia due to anaerobes',
             'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group B','Septicemia due to pseudomonas','Other staphylococcal septicemia',
             'Infection and inflammatory reaction due to other internal orthopedic device, implant, and graft',
             'Other specified bacterial infections in conditions classified elsewhere and of unspecified site, other specified bacteria',
             'Staphylococcus infection in conditions classified elsewhere and of unspecified site, staphylococcus, unspecified',
             'Other specified bacterial infections in conditions classified elsewhere and of unspecified site, other anaerobes',
             'Infection and inflammatory reaction due to other internal prosthetic device, implant, and graft',
             'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group A','Staphylococcal septicemia, unspecified',
             'Infection and inflammatory reaction due to internal joint prosthesis','Septicemia due to gram-negative organism, unspecified','Septicemia due to serratia',
             'Infection and inflammatory reaction due to nervous system device, implant, and graft',
             'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group C',
             'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, unspecified','Infected postoperative seroma',
             'Infection and inflammatory reaction due to peritoneal dialysis catheter','Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group G',
             'Infection and inflammatory reaction due to other genitourinary device, implant, and graft','Gas gangrene']

infection2_ICD9=infection2_ICD9[infection2_ICD9['DIAG_LONG_TITLE'].isin(annot_title)]
infection2_ICD9

,SUBJECT_ID,HADM_ID,DIAG_SEQ_NUM,DIA_ICD9_CODE,DIAG_SHORT_TITLE,DIAG_LONG_TITLE,ICD9,ICD9_NAME,Category
88,98829,109917,13.0,99851,Infected postop seroma,Infected postoperative seroma,998,NaN,Injury and Poisoning
179,75824,121296,1.0,99851,Infected postop seroma,Infected postoperative seroma,998,NaN,Injury and Poisoning
285,26337,132655,3.0,99851,Infected postop seroma,Infected postoperative seroma,998,NaN,Injury and Poisoning
307,63073,134830,11.0,99851,Infected postop seroma,Infected postoperative seroma,998,NaN,Injury and Poisoning
362,58781,141150,3.0,99851,Infected postop seroma,Infected postoperative seroma,998,NaN,Injury and Poisoning
...,...,...,...,...,...,...,...,...,...
4482,27024,198972,7.0,99662,React-oth vasc dev/graft,Infection and inflammatory reaction due to oth...,996,NaN,Injury and Poisoning
4483,4059,198999,1.0,99661,React-cardiac dev/graft,Infection and inflammatory reaction due to car...,996,NaN,Injury and Poisoning
4484,4966,199069,2.0,99662,React-oth vasc dev/graft,Infection and inflammatory reaction due to oth...,996,NaN,Injury and Poisoning
4485,51780,199075,1.0,99665,React-oth genitourin dev,Infection and inflammatory reaction due to oth...,996,NaN,Injury and Poisoning


In [20]:
infection2_ICD9.to_csv('infection2_ICD9.csv',index=False)

In [ ]:
# Postoperative Infection2 (Microbiology)

In [21]:
MICRO_EVENT = pd.read_csv('physionet.org/files/mimiciii/1.4/MICROBIOLOGYEVENTS.csv.gz')

In [22]:
surg_MICRO_EVENT = MICRO_EVENT[MICRO_EVENT['HADM_ID'].isin(X.HADM_ID)] 
surg_MICRO_EVENT = surg_MICRO_EVENT.drop(['ROW_ID','AB_ITEMID','DILUTION_TEXT','DILUTION_COMPARISON',
                                          'DILUTION_VALUE','AB_NAME','ORG_ITEMID','SPEC_ITEMID'],axis=1)

In [24]:
#infection2：patients who has org
infection2=surg_MICRO_EVENT.dropna(subset=['ORG_NAME'])
infection2=infection2.reset_index(drop=True)
infection2

,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_TYPE_DESC,ORG_NAME,ISOLATE_NUM,INTERPRETATION
0,41,101757,2133-01-15 00:00:00,NaN,BRONCHOALVEOLAR LAVAGE,KLEBSIELLA PNEUMONIAE,1.0,I
1,41,101757,2133-01-15 00:00:00,NaN,BRONCHOALVEOLAR LAVAGE,KLEBSIELLA PNEUMONIAE,1.0,S
2,41,101757,2133-01-15 00:00:00,NaN,BRONCHOALVEOLAR LAVAGE,KLEBSIELLA PNEUMONIAE,1.0,S
3,41,101757,2133-01-15 00:00:00,NaN,BRONCHOALVEOLAR LAVAGE,KLEBSIELLA PNEUMONIAE,1.0,S
4,41,101757,2133-01-15 00:00:00,NaN,BRONCHOALVEOLAR LAVAGE,KLEBSIELLA PNEUMONIAE,1.0,S
...,...,...,...,...,...,...,...,...
131233,99783,126090,2125-11-05 00:00:00,2125-11-05 13:15:00,TISSUE,ASPERGILLUS FUMIGATUS,1.0,NaN
131234,99783,126090,2125-11-05 00:00:00,2125-11-05 13:15:00,TISSUE,ASPERGILLUS FUMIGATUS,2.0,NaN
131235,99783,126090,2125-11-05 00:00:00,2125-11-05 13:15:00,TISSUE,ASPERGILLUS FUMIGATUS,2.0,NaN
131236,99783,126090,2125-11-06 00:00:00,2125-11-06 10:24:00,TISSUE,ASPERGILLUS FUMIGATUS,1.0,NaN


In [29]:
#infection3：patient who has org and is a surgical patients
infection3=infection2[infection2.HADM_ID.isin(X.HADM_ID)]
infection3=infection3.drop(['SUBJECT_ID','CHARTDATE','SPEC_TYPE_DESC','ISOLATE_NUM','INTERPRETATION'],axis=1)
infection3=pd.merge(infection3,X,how='outer')

In [31]:
micro_summary = infection3.groupby('ORG_NAME').agg({'HADM_ID':'nunique'}).sort_values(by=['HADM_ID'],ascending=False)
micro_summary = micro_summary.rename(columns={'HADM_ID':'Frequency'})
micro_summary['Sample Size'] = infection3.HADM_ID.nunique()
micro_summary['Percent(%)'] = 100 * micro_summary.Frequency / infection3.HADM_ID.nunique()

In [33]:
annot_org=['STAPH AUREUS COAG +','STAPHYLOCOCCUS, COAGULASE NEGATIVE','ENTEROCOCCUS SP.','ESCHERICHIA COLI','GRAM NEGATIVE ROD(S)','PSEUDOMONAS AERUGINOSA',
           'POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS','ENTEROBACTER CLOACAE','GRAM NEGATIVE ROD #2','PROBABLE ENTEROCOCCUS','ENTEROCOCCUS FAECIUM','GRAM NEGATIVE ROD #3',
           'GRAM NEGATIVE ROD #1','STAPHYLOCOCCUS SPECIES','STAPHYLOCOCCUS EPIDERMIDIS','GRAM NEGATIVE ROD #4','ANAEROBIC GRAM NEGATIVE ROD(S)','STREPTOCOCCUS SPECIES']
infection3=infection3[infection3['ORG_NAME'].isin(annot_org)]
infection3=infection3.drop_duplicates()

In [34]:
infection3

,HADM_ID,CHARTTIME,ORG_NAME,SUBJECT_ID,ADMITTIME,AGE,DISCHTIME,DEATHTIME,INSURANCE,GENDER,...,PainScore_std,PainScore_count,BaselinePainScore_mean,BaselinePainScore_max,BaselinePainScore_min,BaselinePainScore_std,BaselinePainScore_N,Morbidities_N,Surgery,MORTARLITY
15,101757,2133-01-16 10:00:00,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",41,2132-12-31 10:30:00,57,2133-01-27 15:45:00,NaN,Self-Pay/Private,M,...,1.865615,22.0,NaN,NaN,NaN,NaN,NaN,18,Neurologic,0
27,101757,2133-01-03 00:52:00,ESCHERICHIA COLI,41,2132-12-31 10:30:00,57,2133-01-27 15:45:00,NaN,Self-Pay/Private,M,...,1.865615,22.0,NaN,NaN,NaN,NaN,NaN,18,Neurologic,0
40,101757,2133-01-03 00:52:00,STAPH AUREUS COAG +,41,2132-12-31 10:30:00,57,2133-01-27 15:45:00,NaN,Self-Pay/Private,M,...,1.865615,22.0,NaN,NaN,NaN,NaN,NaN,18,Neurologic,0
46,101757,2133-01-03 00:52:00,GRAM NEGATIVE ROD #2,41,2132-12-31 10:30:00,57,2133-01-27 15:45:00,NaN,Self-Pay/Private,M,...,1.865615,22.0,NaN,NaN,NaN,NaN,NaN,18,Neurologic,0
49,101757,2133-01-04 10:41:00,STAPH AUREUS COAG +,41,2132-12-31 10:30:00,57,2133-01-27 15:45:00,NaN,Self-Pay/Private,M,...,1.865615,22.0,NaN,NaN,NaN,NaN,NaN,18,Neurologic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131215,194801,2180-02-21 08:41:00,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",99538,2180-02-13 17:46:00,72,2180-02-26 15:35:00,NaN,Medicare/Government,F,...,2.372586,16.0,NaN,NaN,NaN,NaN,NaN,27,Cardiac,0
131223,144804,2115-02-11 20:30:00,STAPH AUREUS COAG +,99796,2115-02-11 17:27:00,63,2115-02-19 01:30:00,NaN,Medicare/Government,M,...,1.264911,6.0,NaN,NaN,NaN,NaN,NaN,21,Cardiac,0
131229,186518,2142-02-16 22:45:00,GRAM NEGATIVE ROD(S),99814,2142-02-16 16:33:00,87,2142-02-21 16:38:00,NaN,Medicare/Government,F,...,0.000000,2.0,NaN,NaN,NaN,NaN,NaN,11,Circulatory,0
131230,112769,2116-10-13 15:46:00,"STAPHYLOCOCCUS, COAGULASE NEGATIVE",98359,2116-10-11 19:02:00,48,2116-10-20 17:30:00,NaN,Self-Pay/Private,F,...,2.287979,33.0,NaN,NaN,NaN,NaN,NaN,16,General,0


In [41]:
micro_id=infection3.HADM_ID
org_id=infection2.HADM_ID

ICD9infected=X[X.HADM_ID.isin(infection2_ICD9.HADM_ID)]#ICD9 infection = 1
ICD9infected.loc[:, 'infection'] = 1
ICD9_id=ICD9infected.HADM_ID

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [43]:
import copy
icd_micro=copy.deepcopy(X)
icd_micro=pd.DataFrame(data=icd_micro)
icd_micro['infection'] = 0

icd_micro.loc[icd_micro.HADM_ID.isin(micro_id), 'infection'] = 1
icd_micro.loc[icd_micro.HADM_ID.isin(ICD9_id), 'infection'] = 1
icd_micro=icd_micro.reset_index(drop=True)
icd_micro.to_csv('icd_micro.csv',index=False)

In [ ]:
#Preoperation infection

In [47]:
time=pd.merge(service,infection3,how='inner')
time=time.dropna(subset=['CHARTTIME'])
time['TRANSFERTIME'] = pd.to_datetime(time['TRANSFERTIME'])
time['CHARTTIME'] = pd.to_datetime(time['CHARTTIME'])
time['pre_infection'] = np.where(time['TRANSFERTIME'] > time['CHARTTIME'], '1', '0')
time['post_infection'] = np.where(time['TRANSFERTIME'] < time['CHARTTIME'], '1', '0')
pre_infection=time[time['pre_infection']=='1']
post_infection=time[time['post_infection']=='1']

In [53]:
icd_micro['pre_infection']=0
icd_micro['post_infection']=0
all_infected=icd_micro[icd_micro['infection']==1]
unknown_infec=all_infected[~all_infected['HADM_ID'].isin(time['HADM_ID'])]
unknown_infecID=unknown_infec.HADM_ID

In [56]:
icd_micro.loc[icd_micro['HADM_ID'].isin(pre_infection['HADM_ID']), 'pre_infection'] = 1
icd_micro.loc[icd_micro['HADM_ID'].isin(post_infection['HADM_ID']), 'post_infection'] = 1
icd_micro.loc[icd_micro['HADM_ID'].isin(unknown_infecID), 'post_infection'] = 'NA'
icd_micro.loc[icd_micro['HADM_ID'].isin(unknown_infecID), 'pre_infection'] = 'NA'

icd_micro.to_csv('icd_micro.csv',index=False)

In [ ]:
icd_micro

,SUBJECT_ID,HADM_ID,ADMITTIME,AGE,DISCHTIME,DEATHTIME,INSURANCE,GENDER,ETHNICITY,RELIGION,...,BaselinePainScore_max,BaselinePainScore_min,BaselinePainScore_std,BaselinePainScore_N,Morbidities_N,Surgery,MORTARLITY,infection,pre_infection,post_infection
0,23,152223,2153-09-03 07:15:00,71,2153-09-08 19:10:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,8,Cardiac,0,0,0,0
1,23,124321,2157-10-18 19:34:00,75,2157-10-25 14:00:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,10,Neurologic,0,0,0,0
2,28,162569,2177-09-01 07:15:00,74,2177-09-06 16:00:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,10,Cardiac,0,0,0,0
3,32,175413,2170-04-04 08:00:00,46,2170-04-23 12:45:00,NaN,Medicaid,M,WHITE,UNOBTAINABLE,...,NaN,NaN,NaN,NaN,6,Thoracic,0,0,0,0
4,35,166707,2122-02-10 11:15:00,69,2122-02-20 15:30:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,11,Cardiac,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20060,98761,182540,2186-02-08 12:30:00,47,2186-02-08 20:21:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,14,General,0,0,0,0
20061,98769,141860,2125-02-22 22:18:00,80,2125-03-05 17:22:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,15,General,0,1,0,1
20062,98790,187987,2200-10-15 01:58:00,54,2200-10-25 15:43:00,NaN,Self-Pay/Private,M,WHITE,NOT SPECIFIED,...,NaN,NaN,NaN,NaN,10,Musculoskeletal,0,1,NA,NA
20063,98794,190603,2127-11-07 11:00:00,78,2127-11-11 14:30:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,NaN,NaN,NaN,NaN,5,Cardiac,0,0,0,0


In [58]:
#postoperative infection
((icd_micro['pre_infection'] == 0) & (icd_micro['post_infection'] == 1)).sum()

3602

In [59]:
#preoperative infection
((icd_micro['pre_infection'] == 1) & (icd_micro['post_infection'] == 0)).sum()

167

In [60]:
#postoperative infection & preoperative infection
((icd_micro['pre_infection'] == 1) & (icd_micro['post_infection'] == 1)).sum()

1454

In [61]:
#infected but unknown time
((icd_micro['pre_infection'] == 'NA') & (icd_micro['post_infection'] == 'NA')).sum()

816

In [ ]:
#merge dataset

In [64]:
final_infec_cormor=pd.merge(icd_micro,morb,how='outer')
final_infec_cormor

,SUBJECT_ID,HADM_ID,ADMITTIME,AGE,DISCHTIME,DEATHTIME,INSURANCE,GENDER,ETHNICITY,RELIGION,...,Cormor_Diabetes,Cormor_Fluid and electrolyte disorders,Cormor_Hypertension,Cormor_Liver Disease,Cormor_Metastatic Carcinoma,Cormor_Myocardial Infarction (heart attack),Cormor_Obesity,Cormor_Peripheral Vascular Disease,Cormor_Valvular Disease,Cormor_Weight Loss
0,23,152223,2153-09-03 07:15:00,71,2153-09-08 19:10:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,124321,2157-10-18 19:34:00,75,2157-10-25 14:00:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28,162569,2177-09-01 07:15:00,74,2177-09-06 16:00:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32,175413,2170-04-04 08:00:00,46,2170-04-23 12:45:00,NaN,Medicaid,M,WHITE,UNOBTAINABLE,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,35,166707,2122-02-10 11:15:00,69,2122-02-20 15:30:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20060,98761,182540,2186-02-08 12:30:00,47,2186-02-08 20:21:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20061,98769,141860,2125-02-22 22:18:00,80,2125-03-05 17:22:00,NaN,Medicare/Government,F,WHITE,CATHOLIC,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20062,98790,187987,2200-10-15 01:58:00,54,2200-10-25 15:43:00,NaN,Self-Pay/Private,M,WHITE,NOT SPECIFIED,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20063,98794,190603,2127-11-07 11:00:00,78,2127-11-11 14:30:00,NaN,Medicare/Government,M,WHITE,CATHOLIC,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
final_infec_cormor.to_csv('final_infec_cormor.csv',index=False)